In [169]:
import yfinance as yf

data = yf.download(tickers = "GC=F", start="2017-12-01", end="2023-06-11", interval='1d')

[*********************100%***********************]  1 of 1 completed


In [170]:
data = data.reset_index('Date', inplace=False)

In [171]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-12-01,1274.099976,1288.599976,1271.000000,1278.800049,1278.800049,823
1,2017-12-04,1272.500000,1276.099976,1270.300049,1274.300049,1274.300049,850
2,2017-12-05,1275.400024,1275.800049,1260.000000,1261.599976,1261.599976,1499
3,2017-12-06,1264.900024,1267.800049,1261.599976,1262.800049,1262.800049,373
4,2017-12-07,1262.300049,1262.400024,1243.900024,1249.800049,1249.800049,358
...,...,...,...,...,...,...,...
1385,2023-06-05,1947.500000,1961.900024,1937.800049,1958.000000,1958.000000,713
1386,2023-06-06,1960.800049,1965.500000,1958.800049,1965.500000,1965.500000,164
1387,2023-06-07,1959.000000,1968.400024,1940.699951,1942.699951,1942.699951,139
1388,2023-06-08,1943.199951,1969.000000,1943.099976,1963.599976,1963.599976,433


In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional
from sklearn.ensemble import RandomForestRegressor

In [173]:
data['Date'] = pd.to_datetime(data['Date'])

In [174]:
features = data[['Date']]
labels = data['Close']

In [175]:
from sklearn.model_selection import train_test_split
train_features, test_valid_features, train_labels, test_valid_labels = train_test_split(features, labels, test_size=0.3, random_state=42)

# Chia tập kiểm tra + xác thực thành tập kiểm tra và tập xác thực
test_features, valid_features, test_labels, valid_labels = train_test_split(test_valid_features, test_valid_labels, test_size=0.67, random_state=42)


train_size = int(0.7 * len(data))
test_size = int(0.2 * len(data))
val_size = len(data) - train_size - test_size

train_data = data[:train_size]
test_data = data[train_size:train_size+test_size]
val_data = data[train_size+test_size:]

x_train = train_data[['Date']]
y_train = train_data['Close']

x_test = test_data[['Date']]
y_test = test_data['Close']

x_val = val_data[['Date']]
y_val = val_data['Close']

In [176]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

best_score = float('-inf')
best_params = {}

for n_estimators in [100, 200, 500, 1000]:
    for max_depth in [5, 10, 15, 20]:
        for random_state in [1, 42, 77, 123]:
            model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state, max_depth=max_depth)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
            score = model.score(x_test, y_test)  # R-squared score
            
            if score > best_score:
                best_score = score
                best_params = {
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'random_state': random_state
                }

print("Best parameters:", best_params)

Best parameters: {'n_estimators': 100, 'max_depth': 5, 'random_state': 42}


In [178]:
model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)
model.fit(train_features, train_labels)

RandomForestRegressor(max_depth=5, random_state=42)

In [179]:
y_pred = model.predict(test_features)

In [181]:
y_val = model.predict(valid_features)

In [185]:
df = pd.DataFrame({'x': y_pred, 'y': test_labels})

# In bảng
print(df)

                x            y
712   1875.525581  1900.199951
513   1475.113386  1472.599976
591   1718.371389  1744.800049
560   1637.174545  1640.000000
411   1424.553905  1413.900024
...           ...          ...
894   1822.108800  1775.599976
63    1328.416144  1326.000000
277   1293.565092  1287.099976
1185  1752.318853  1755.300049
371   1295.699267  1280.599976

[137 rows x 2 columns]
